In [3]:
# https://carla.readthedocs.io/en/0.9.12/adv_agents/#:~:text=set_destination(end_location%2C%20start_location%3DNone,use%20the%20current%20agent%20location
#BEHAVIOUR AGENT CON SETTAGGIO DESTINAZIONE

#all imports
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys

from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

#colora di bianco il percorso dato in input
def draw_entire_route(route, duration):
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=True, color=carla.Color(r=0, g=0, b=255), life_time=duration)

def erase_route(route, duration):
    for waypoint in route:
        world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=True, color=carla.Color(r=255, g=255, b=255), life_time=duration)

def draw_symbol(wp, duration, symbol, r, g, b):
    world.debug.draw_string(wp.transform.location, symbol, draw_shadow=False, color=carla.Color(r, g, b), life_time=duration)


#DISEGNA DISEGNA 10 SIMBOLI DI FRONTE IL VEICOLO LUNGO IL PERCORSO
def draw_route(wp, route,seconds=3.0): 
    if len(route)-wp <25: # A 25 PUNTI DALLA FINE COLORA I WP DI ROSSO
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=255, b=0) #ALTRIMENTI DI VERDE
    for i in range(10): #STAMPA 10 SIMBOLI DI FRONTE AL VEICOLO IN BASE AL COLORE PRECEDEMENTE SCELTO
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False, color=draw_colour, life_time=seconds, persistent_lines=True)

######################################################################################################################################################
#########################################################            MAIN CODE               #########################################################
######################################################################################################################################################

# CONNESSIONE AL SIMULATORE
client = carla.Client('localhost', 2000)

#RECUPERA ASSETS DEL SIMULATORE
world = client.get_world()
bp_lib = world.get_blueprint_library() 

#TUTTI I PUNTI DI SPAWN (CONSIGLIATI) DELLA MAPPA
spawn_points = world.get_map().get_spawn_points()

# GENERA NV AUTO CASUALI ABILITANDONE L'AUTOPILOT DI DEFAULT 
NV = 25
for i in range(NV): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True)

# ISTANZIA L'EGO VEHICLE IN UN PUNTO A CASO DELLA MAPPA
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
start_spawn = random.choice(spawn_points)
vehicle = world.try_spawn_actor(vehicle_bp, start_spawn)

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

#RECUPERO I WAYPOINTS CORRISPONDENTI RISPETTIVAMENTE ALLE COORDINATE DI SPWN DI SPAWN DEL VEICOLO E DI DESTINAZIONE DELL'AGENTE
start_location = vehicle.get_location()
start_waypoint = world.get_map().get_waypoint(start_location)

#PUNTO A CASO DALLA MAPPA COME DESTINAZIONE, SCEGLIE DI DEFAULT IL PERCORSO PIU' BREVE VERSO QUEL PUNTO
dest_location = random.choice(spawn_points).location
dest_waypoint = world.get_map().get_waypoint(dest_location)

#L' AGENTE BEHAVIOUR REAGISCE A SEMAFORI E SEGNALI DI STOP, EVITA PEDONI, SEGUE LE MACCHINE 
#3 POSSIBILI PROFILI AL MOMENTO: cautious, normal, aggressive -> https://carla.readthedocs.io/en/0.9.12/adv_agents/#behavior-types
agent = BehaviorAgent(vehicle, behavior='cautious')

#RECUPERA IL PERCORSO CHE SEGUIRA' L'AGENTE IN UNA LISTA DI WAYPOINTS
route = agent.trace_route(start_waypoint, dest_waypoint)
draw_entire_route(route, 120)
draw_symbol(start_waypoint, 120, '<<<<<!!!!!-START-!!!!!>>>>>', 0, 255, 0)
draw_symbol(dest_waypoint, 120, '<<<<<!!!!!-END-!!!!!>>>>>', 255, 0, 0)


#DESTINAZIONE AGENT
agent.set_destination(dest_location)

#MAIN LOOP:
while True:
    if agent.done(): #SE L'AGENT HA CONCLUSO COLORA DI BIANCO IL PERCORSO COMPLETATO
        print("The target has been reached, stopping the simulation")
        #COLORA DI BIANCO I SIMBOLI DEL PERCORSO COMPLETATO
        erase_route(route, 60)
        draw_symbol(start_waypoint, 60, '<<<<<!!!!!-START-!!!!!>>>>>', 255, 255, 255)
        draw_symbol(dest_waypoint, 60, '<<<<<!!!!!-END-!!!!!>>>>>', 255, 255, 255)
        break
        
    vehicle.apply_control(agent.run_step())
    curr_wp = world.get_map().get_waypoint(vehicle.get_location())
    draw_symbol(curr_wp, 0.01, '!!!CAR!!!', 255, 255, 255) 


print("SIMULATION COMPLETED")

#DISTRUGGE TUTTI I VEICOLI DELLA SIMULAZIONE 
for v in world.get_actors().filter('*vehicle*'): 
    v.destroy()


KeyboardInterrupt: 